In [1]:
import os
import pandas as pd
import json
import glob

In [2]:
city_list=os.listdir("C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather")
file_path="C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather\\"
weather_list=[]
for p in city_list:
    json_list=os.listdir(file_path+p)
    s=json_list.index('2016-1.json')
    actual_list=json_list[s:]
    for i in actual_list:
        fp=open(file_path+p+"\\"+i,"r")
        d=fp.read()
        data=json.loads(d)
        feature_list=['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph',         'tempF','WindChillF','humidity','time']
        
        for i in data["data"]["weather"]:
            for j in range(24):
                weather_per_hr=dict()
                for k in feature_list:
                    weather_per_hr[k]=i["hourly"][j][k]
                weather_per_hr['date']=i['date']
                weather_per_hr['airport']=data["data"]["request"][0]["query"]
                weather_list.append(weather_per_hr)
        fp.close()
df_weather=pd.DataFrame(weather_list)

df_weather.to_csv("weather_features.csv")

In [3]:
df_weather["air"]=""

In [4]:
df_weather["air"]=[x[0:3] for x in df_weather["airport"]]

In [5]:
df_weather["int_time"]=0

In [6]:
df_weather["int_time"]=[int(x) for x in df_weather["time"]]


In [2]:
files=glob.glob('C:\\Users\\Abi\\Desktop\\ML_project\\Flight Data-20200805T074725Z-001\\Flight Data\**\*',recursive=True)
df_flight=pd.DataFrame()
for i in files:
    try:
        i.index(".csv")
    except:
        continue
    d=pd.read_csv(i)
    d=d[['FlightDate','Quarter','Year','Month'
    ,'DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15',           'ArrDelayMinutes','Origin','Dest']]
    df_flight=pd.concat([df_flight,d])

    

In [8]:
airports=['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
df_flight=df_flight[df_flight["Dest"].isin(airports)]
df_flight=df_flight[df_flight["Origin"].isin(airports)]





In [9]:
df_flight=df_flight.dropna()

In [10]:
df_flight["round_time"]=0

In [11]:
df_flight["round_time"]=[(round((x/100))*100)%2400 for x in df_flight["CRSDepTime"] ]

In [12]:
result=pd.merge(left=df_weather,right=df_flight,left_on=["date","int_time","air"],right_on=["FlightDate","round_time","Origin"])

In [13]:
del result["int_time"]
del result["air"]
del result["round_time"]
del result["Origin"]
del result["Dest"]

In [16]:
result=result.dropna()